In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train_df = pd.read_csv('/content/drive/My Drive/cone/ConoTrainingData.csv')

In [3]:
train_df.head(2)

,CLASS LABEL,SEQUENCE,ACC_ID
0,Ca,GCCSHPACSVNHPELC,Q1L777
1,Ca,CYDGGTSCDSGIQCCSGWCIFVCL,Q9U648


In [4]:
test_df = pd.read_csv('/content/drive/My Drive/cone/ConoTestData.csv')

In [5]:
test_df.head(2)

,CLASS LABEL,SEQUENCE,ACC_ID
0,K,QKSLVPSVITTCCGYDPGTMCPPCRCTNSC,P0C828
1,K,SRCFPPGIYCTPYLPCCWGICCGTCRNVCHLRI,Q7YZS9


In [6]:
train_df.describe()

,CLASS LABEL,SEQUENCE,ACC_ID
count,145,145,145
unique,3,145,145
top,Ca,GCCSHPACSVNHPELC,Q1L777
freq,70,1,1


In [7]:
sum(train_df['CLASS LABEL']=='Ca')

70

In [8]:
sum(train_df['CLASS LABEL']=='K')

26

In [9]:
sum(train_df['CLASS LABEL']=='Na')

49

In [10]:
train_df['CLASS LABEL'].unique()

array(['Ca', 'K', 'Na'], dtype=object)

In [11]:
test_df.describe()

,CLASS LABEL,SEQUENCE,ACC_ID
count,70,70,70
unique,3,67,70
top,Na,GHVPCGKDGRKCGYHADCCNCCLSGICKPSTSWTGCSTSTFD,P0C828
freq,37,2,1


In [12]:
sum(test_df['CLASS LABEL']=='Ca')


21

In [13]:
sum(test_df['CLASS LABEL']=='K')


12

In [14]:
sum(test_df['CLASS LABEL']=='Na')

37

In [15]:
!pip install fair-esm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.8 MB/s eta 0:00:00


In [16]:
import torch
import esm

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [17]:
train_data = [(tuple(list(x))[1], tuple(list(x))[0])  for x in list(train_df[['SEQUENCE','ACC_ID']].values)]

In [18]:
dummy_train = train_data

In [19]:
tr_batch_labels, tr_batch_strs, tr_batch_tokens = batch_converter(dummy_train)
tr_batch_lens = (tr_batch_tokens != alphabet.padding_idx).sum(1)



In [20]:
# Extract per-residue representations (on CPU)
with torch.no_grad():
    tr_results = model(tr_batch_tokens, repr_layers=[33], return_contacts=False)




In [21]:
tr_token_representations = tr_results["representations"][33]

In [22]:
# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
tr_sequence_representations = []
for i, tokens_len in enumerate(tr_batch_lens):
    tr_sequence_representations.append(tr_token_representations[i, 1 : tokens_len - 1].mean(0))


this is the length of each string + 2 (beginning and end)

the letters start with 0, end with 2, and 1 is used for padding. Because we have 20 different amino acids, we have 23 different numbers.

In [23]:
train_labels = train_df['CLASS LABEL']

In [24]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
label_encoder = OneHotEncoder()


In [25]:
train_labels = train_labels

In [26]:

tr_encoded_data = label_encoder.fit_transform(train_labels.values.reshape(-1,1))

In [27]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=15, random_state=42)

In [28]:
x = [tensor.numpy() for tensor in tr_sequence_representations]

In [29]:
# begin smote



from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from collections import Counter

sm_1 = SMOTE(random_state=42)
X_res, y_res = sm_1.fit_resample(x,  tr_encoded_data.toarray())



In [30]:
rfc.fit(X_res, y_res)

RandomForestClassifier(n_estimators=15, random_state=42)

In [31]:
y_pred = rfc.predict(X_res)

In [34]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
accuracy = accuracy_score(y_res, y_pred)
precision = precision_score(y_res, y_pred, average='macro')
recall = recall_score(y_res, y_pred, average='macro')
f1 = f1_score(y_res, y_pred, average='macro')


train

In [35]:
test_data = [(tuple(list(x))[1], tuple(list(x))[0])  for x in list(test_df[['SEQUENCE','ACC_ID']].values)]
test_batch_labels, test_batch_strs, test_batch_tokens = batch_converter(test_data)
test_batch_lens = (test_batch_tokens != alphabet.padding_idx).sum(1)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    test_results = model(test_batch_tokens, repr_layers=[33], return_contacts=False)
test_token_representations = test_results["representations"][33]

In [36]:




# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
test_sequence_representations = []
for i, tokens_len in enumerate(test_batch_lens):
    test_sequence_representations.append(test_token_representations[i, 1 : tokens_len - 1].mean(0))
test_labels = test_df['CLASS LABEL']


In [37]:
transformed_labels =label_encoder.transform(test_labels.values.reshape(-1,1))

In [38]:
x_test = [tensor.numpy() for tensor in test_sequence_representations]

In [39]:
y_pred_test = rfc.predict(x_test)

In [40]:
def func_to_check(max_dep, n_estimators):
  rfc = RandomForestClassifier(n_estimators=n_estimators, random_state=42, max_depth=max_dep)
  x = [tensor.numpy() for tensor in tr_sequence_representations]
  rfc.fit(x, tr_encoded_data.toarray())
  y_pred = rfc.predict(x)
  accuracy = accuracy_score(tr_encoded_data.toarray(), y_pred)
  precision = precision_score(tr_encoded_data.toarray(), y_pred, average='macro')
  recall = recall_score(tr_encoded_data.toarray(), y_pred, average='macro')
  f1 = f1_score(tr_encoded_data.toarray(), y_pred, average='macro')
  y_pred_test = rfc.predict(x_test)
  test_accuracy = accuracy_score(transformed_labels.toarray(), y_pred_test)
  test_precision = precision_score(transformed_labels.toarray(), y_pred_test, average='weighted')
  test_recall = recall_score(transformed_labels.toarray(), y_pred_test, average='weighted')
  test_f1 = f1_score(transformed_labels.toarray(), y_pred_test, average='weighted')
  return accuracy, precision, recall, f1 , test_accuracy, test_precision, test_recall, test_f1



In [41]:
import numpy as np
n_estimators_range = np.linspace(10, 200, 10).astype(int)

printed metrics are training accuracy, training precision, training recall, training f1 score, testing accuracy, testing precision, testing recall, testing f1 score. 

In [42]:
for n_estimators in n_estimators_range:
  for max_dep in [5, 7, 8, 10, 15, 20, 25, 30, 40]:
  
    #max_depth.append(max_dep)
    #n_estimators_1.append(n_estimators)
    #train_acc_1, train_prec_1, train_recall_1, train_f1_1, test_acc_1, test_prec_1, test_recall_1, test_f1_1 = func_to_check(max_dep, n_estimators)
    #train_acc.append(train_acc_1)
    #train_prec.append(train_prec_1)
    #train_recall.append(train_recall_1)
    #train_f1.append(train_f1_1)
    #test_acc.append(test_acc_1)
    #test_prec.append(test_prec_1)
    #test_recall.append(test_recall_1)
    #test_f1.append(test_f1_1)
        

    print("max depth: ", max_dep, "n_ estimators:", n_estimators, "metrics:", func_to_check(max_dep, n_estimators))

max depth:  5 n_ estimators: 10 metrics: (0.9517241379310345, 1.0, 0.9303506017791733, 0.9628796962430055, 0.8285714285714286, 0.9285714285714286, 0.8285714285714286, 0.865952380952381)
max depth:  7 n_ estimators: 10 metrics: (0.9793103448275862, 1.0, 0.9776556776556777, 0.9886331344131856, 0.8285714285714286, 0.9428571428571428, 0.8285714285714286, 0.8746674378253325)
max depth:  8 n_ estimators: 10 metrics: (0.9793103448275862, 1.0, 0.9776556776556777, 0.9886331344131856, 0.7857142857142857, 0.984453781512605, 0.7857142857142857, 0.8606094772972199)
max depth:  10 n_ estimators: 10 metrics: (0.9793103448275862, 1.0, 0.9776556776556777, 0.9886331344131856, 0.7857142857142857, 0.984453781512605, 0.7857142857142857, 0.8606094772972199)
max depth:  15 n_ estimators: 10 metrics: (0.9793103448275862, 1.0, 0.9776556776556777, 0.9886331344131856, 0.7857142857142857, 0.984453781512605, 0.7857142857142857, 0.8606094772972199)
max depth:  20 n_ estimators: 10 metrics: (0.9793103448275862, 1.0,

Seems like one of the best parameters and metrics are: 



max depth:  7 n_ estimators: 115 metrics: (1.0, 1.0, 1.0, 1.0, 0.9142857142857143, 0.9828571428571428, 0.9142857142857143, 0.9468315559778974)
